In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
import fitsio
import statsmodels.api as sm

In [ ]:
def rlm_fit1d(x, y, t=1.5, order=1):
    '''
    1D robust polynomial fit.
    
    Given x array and y array, calculate the 1D robust 
    polynomial fit of arbitrary order. Huber weight
    function is used. 
    
    See also poly_val1d.py
    
    INPUT:
    1D arrays of x and y values; tunning parameter t; 
    order of the polynomial fit.
    
    OUTPUT:
    Array of parameters of the polynomial [a0, a1, a2 ...] 
    so that y = a0 + a1*x + a2*x**2 + ...
    '''
    

    ncols = order+1
    a = np.zeros((x.size,ncols))
    for i in range(order+1):
        a[:,i] = x**i
    res = sm.RLM(y, a, M=sm.robust.norms.HuberT(t=t)).fit()
    m = res.params
    return m

def poly_val1d(x, m):
    '''
    Evaluate the 1D polynomial from x values and polynomial parameters
    
    See also rlm_fit1d.py
    
    INPUT:
    1D array of x values; 
    1D array of polynomial parameters (for example generated by 
    rlm_fit1d.py).
    
    OUTPUT:
    1D array of the evaluated values of the polynomial.
    '''
    
    order = len(m)-1
    z = np.zeros(x.shape)
    for i in range(order+1):
        z += m[i] * x**i
    return z

In [ ]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [ ]:
img = fitsio.read('/Users/rongpu/Downloads/wiro/20220929/a070.fit')

# Pick a star from DS9
y, x = 2909, 2483
plt.figure(figsize=(15, 15))
plt.imshow(img, cmap='gray', vmin=500, vmax=1000)
plt.plot(y, x, 'x', color='r', ms=10, alpha=0.5)
plt.show()

size = 50
img1 = img[x-size//2:x+size//2, y-size//2:y+size//2]
print(img1.max())

median_sky = np.median(np.concatenate([img1[:, :10].flatten(), img1[:, -20:].flatten(), img1[:10].flatten(), img1[-20:].flatten()]))
img1 = img1 - median_sky

plt.figure(figsize=(6, 6))
plt.imshow(img1, cmap='gray', vmin=-100, vmax=500)
plt.show()

nea = np.sum(img1)**2/np.sum(img1**2)
fwhm = np.sqrt(nea / (4 * np.pi)) * 2.3548
print(fwhm)

In [ ]:
# g band filter
fns = ['a'+str(index).zfill(3)+'.fit' for index in np.arange(67, 75)]
focuses = [-2.2, -2.3, -2.4, -2.5, -2.6, -2.7, -2.8, -2.9]
# fns.remove('a010.fit')
# fns.remove('a015.fit')
print(fns)
fwhm_list = []
focus_list = []
for index, fn in enumerate(fns):
    print(fn)
    img = fitsio.read('/Users/rongpu/Downloads/wiro/20220929/'+fn)
    header = fitsio.read_header('/Users/rongpu/Downloads/wiro/20220929/'+fn)
    print(header['INSTFOCU'], header['CMDFOCUS'])
    # focus = float(header['INSTFOCU'])
    focus = focuses[index]
    
    y, x = 2909, 2483
    size = 50
    img1 = img[x-size//2:x+size//2, y-size//2:y+size//2]

    median_sky = np.median(np.concatenate([img1[:, :10].flatten(), img1[:, -20:].flatten(), img1[:10].flatten(), img1[-20:].flatten()]))
    img1 = img1 - median_sky
    print(np.max(img1))

    median_sky = np.median(np.concatenate([img1[:, :10].flatten(), img1[:, -20:].flatten(), img1[:10].flatten(), img1[-20:].flatten()]))
    img1 = img1 - median_sky

    plt.figure(figsize=(5, 5))
    plt.imshow(img1, cmap='gray', vmin=-50, vmax=1000)
    plt.show()

    nea = np.sum(img1)**2/np.sum(img1**2)
    fwhm = np.sqrt(nea / (4 * np.pi)) * 2.3548
    print(fwhm)
    fwhm_list.append(fwhm)
    focus_list.append(focus)

In [ ]:
focus_list, fwhm_list = np.array(focus_list), np.array(fwhm_list)
plt.plot(focus_list, fwhm_list, '.')
plt.xlabel('Focus adjustment (mm)')
plt.ylabel('FWHM (in pixels)')
plt.show()

In [ ]:
mask = focus_list>-3
mask &= focus_list<-2
x = focus_list[mask]
y = fwhm_list[mask]
plt.plot(x, y, '.')
plt.xlabel('Focus adjustment (mm)')
plt.ylabel('FWHM (in pixels)')
plt.show()

In [ ]:
res = rlm_fit1d(x, y, t=1, order=2)
# print(res)

x_plot = np.linspace(x.min()-0.1, x.max()+0.1, 10000)
y_plot = poly_val1d(x_plot, res)
best_focus = x_plot[np.argmin(y_plot)]
print('Best focus: {:.2f}'.format(best_focus))
plt.plot(x, y, '.')
plt.plot(x_plot, y_plot)
plt.axvline(best_focus)
# plt.xlim(0, 60)
plt.xlabel('Focus adjustment (mm)')
plt.ylabel('FWHM (in pixels)')
plt.title('g filter best focus: {:.2f}'.format(best_focus))
plt.show()

__Humidity and temperature: 61% 46F__